In [2]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [3]:
version1_path = "../data/raw/td_V1.db"
version2_path = "../data/raw/td_V2.db"
con1 = sqlite3.connect(version1_path)
con2 = sqlite3.connect(version2_path)

In [4]:
light_q = """SELECT key
     FROM jira_issues"""

heavy_q = """SELECT *
     FROM jira_issues"""

commits_q = """SELECT *
    FROM git_commits"""

In [5]:
light1 = pd.read_sql(light_q, con1)
light2 = pd.read_sql(light_q, con2)

In [6]:
heavy1 = pd.read_sql(heavy_q, con1)
heavy2 = pd.read_sql(heavy_q, con2)

In [37]:
heavy1.columns

Index(['projectID', 'key', 'creationDate', 'resolutionDate', 'updateDate',
       'dueDate', 'resolution', 'type', 'priority', 'fixVersions', 'versions',
       'timeSpent', 'aggregatedTimeSpent', 'timeEstimate',
       'timeOriginalEstimate', 'aggregateTimeEstimate', 'progressPercent',
       'componentName', 'componentDescription', 'description', 'summary',
       'watchCount', 'votes', 'labels', 'creatorName', 'creatorActive',
       'assignee', 'reporter'],
      dtype='object')

In [33]:
heavy2.columns

Index(['PROJECT_ID', 'KEY', 'CREATION_DATE', 'RESOLUTION_DATE', 'HASH',
       'COMMIT_DATE'],
      dtype='object')

In [7]:
light1.shape, light2.shape

((67427, 1), (22007, 1))

In [8]:
heavy1.shape, heavy2.shape

((67427, 28), (22007, 6))

In [17]:
len(set(light1["key"]).intersection(set(light2["KEY"])))

9300

In [25]:
commits1 = pd.read_sql(commits_q, con1)
commits2 = pd.read_sql(commits_q, con2)

In [28]:
commits1.shape, commits2.shape

((140687, 13), (81072, 13))

In [29]:
len(set(commits1["commitHash"]).intersection(set(commits2["COMMIT_HASH"])))

75009

In [30]:
commits1.columns

Index(['projectID', 'commitHash', 'commitMessage', 'author', 'authorDate',
       'authorTimezone', 'committer', 'committerDate', 'committerTimezone',
       'branches', 'inMainBranch', 'merge', 'parents'],
      dtype='object')